In [1]:
# filter some warning messages
import warnings 
warnings.filterwarnings("ignore") 

#libraries
import datetime as dt
import xarray as xr
import fsspec
import s3fs
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# make datasets display nicely
xr.set_options(display_style="html")  
import os.path

#magic fncts #put static images of your plot embedded in the notebook
%matplotlib inline  
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 


In [ ]:
from dask.distributed import Client, progress
from dask_gateway import Gateway
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.scale(200)
cluster

In [ ]:
%%time
file_location = 's3://mur-sst/zarr-v1'
ikey = fsspec.get_mapper(file_location, anon=True)
ds_sst = xr.open_zarr(ikey,consolidated=True)
ds_sst = ds_sst.sel(time=slice('2015-01-01','2020-01-01'))

In [ ]:
small_subset = ds_sst.sel(lon=slice(150,165),lat=slice(25,50))
small_subset.analysed_sst[0,:,:].plot()
#small_subset

In [ ]:
sm_clim = small_subset.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
sm_anom = small_subset.groupby('time.dayofyear')-sm_clim
ss = sm_anom.analysed_sst.std(dim=('lat','lon')).load()

In [ ]:
plt.plot(ss.time,ss)

In [ ]:
tem = ss.sel(time=slice('2018-07-09','2018-07-15'))
plt.plot(tem.time,tem)

In [ ]:
tem = ds_sst.analysed_sst.sel(lat=slice(0,65),time='2015-08-27T09').load()
tem.coords['lon'] = np.mod(tem['lon'], 360)
tem = tem.sortby(tem.lon)
a1 = tem.sel(lon=slice(100,270))

tem = ds_sst.analysed_sst.sel(lat=slice(0,65),time='2016-08-26T09').load()
tem.coords['lon'] = np.mod(tem['lon'], 360)
tem = tem.sortby(tem.lon)
a2 = tem.sel(lon=slice(100,270))

tem = ds_sst.analysed_sst.sel(lat=slice(0,65),time='2018-07-11T09').load()
tem.coords['lon'] = np.mod(tem['lon'], 360)
tem = tem.sortby(tem.lon)
a3 = tem.sel(lon=slice(100,270))


In [ ]:
#a1 = ds_sst.analysed_sst.sel(lon=slice(100,270),lat=slice(0,65),time='2015-08-27T09').load()
#a2 = ds_sst.analysed_sst.sel(lon=slice(100,270),lat=slice(0,65),time='2016-08-28T09').load()
#a3 = ds_sst.analysed_sst.sel(lon=slice(100,270),lat=slice(0,65),time='2018-07-11T09').load()

In [ ]:
a1.plot(vmin=275,vmax=305,cmap='jet')
plt.savefig('./mursst_20150827.png')   

In [ ]:
a2.plot(vmin=275,vmax=305,cmap='jet')
plt.savefig('./mursst_20160828.png')     

In [ ]:
a3.plot(vmin=275,vmax=305,cmap='jet')
plt.savefig('./mursst_20180711.png')     

In [ ]:
big_subset = ds_sst.sel(lon=slice(130,165),lat=slice(20,50))
bg_clim = big_subset.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
bg_anom = big_subset.groupby('time.dayofyear')-bg_clim

In [ ]:
bg_anom.analysed_sst.plot(vmin=-2,vmax=2,cmap='seismic')